In [1]:
%pip install "huggingface_hub[hf_xet]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 15.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install tf-keras --upgrade

  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.19.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (21 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.19.0-cp311-cp311-macosx_12_0_arm64.whl (252.6 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
Using cached ml_dtypes-0.5.1-cp311-cp311-macosx_10_9_universal2.whl (671 kB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninst

In [2]:
%pip install "numpy<2.0" --force-reinstall


  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
from joblib import dump
from sentence_transformers import SentenceTransformer


/Users/m.mughees/Desktop/Pi515-AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from data_preparation.Transparency_data_preparation import (
    create_transparency_pipeline,
    prepare_am_transparency_data,
    prepare_pm_transparency_data
)

In [3]:
def evaluate_xgb(X_train, y_train, X_dev, y_dev):
    print("Evaluating XGBoost Regressor...")

    # Define the hyperparameter grid search to try combinations of these hyperparameters.
    param_grid = {
        'algo__n_estimators': [1000],
        'algo__max_depth': [2, 3, 4],
        'algo__learning_rate': [0.01, 0.05, 0.1], # smaller learning rate is possibly better as training consisitency increasees.
        'algo__subsample': [0.8, 1.0],
    }

    # This here uses the pipeline to handle missing values, scaling, encoding, etc for teh dataset. hi
    pipeline = create_transparency_pipeline()

    # This combines the preprocessing and XGBoost model into one clean pipeline.
    pipeline_with_algo = Pipeline(steps=[
        ('preprocessor', pipeline),
        ('algo', XGBRegressor(
            objective='reg:squarederror',
            random_state=42
        ))
    ])

    grid_search = GridSearchCV(
        pipeline_with_algo, param_grid,
        cv=5,  # 5-fold cross-validation
        scoring='neg_mean_squared_error',  
        verbose=1  # Show progress in terminal
    )
    grid_search.fit(X_train, y_train)

    # This shows us our best model based on cross-validation R² score.
    best_estimator = grid_search.best_estimator_

    # 📊 FEATURE IMPORTANCE SECTION
    try:
        model = best_estimator.named_steps["algo"]
        preprocessor = best_estimator.named_steps["preprocessor"]
        feature_names = preprocessor.get_feature_names_out()
        importances = model.feature_importances_

        feature_df = pd.DataFrame({
            "Feature": feature_names,
            "Importance": importances
        }).sort_values(by="Importance", ascending=False)

        print("\nTop 10 Most Important Features:")
        print(feature_df.head(10))
    except Exception as e:
        print("Could not extract feature importances:", e)

    # We are making predicitons on the dev set here
    y_pred = best_estimator.predict(X_dev)

    # Here we are calculating the following values
    # Calculate evaluation metrics
    rmse = np.sqrt(mean_squared_error(y_dev, y_pred))
    mape = mean_absolute_percentage_error(y_dev, y_pred)
    r2 = r2_score(y_dev, y_pred)

    # Shows you the best performance from the training phase and the hyperparameters that gave it.
    print("Grid searching is done!")
    print("Best score (neg MSE):", grid_search.best_score_)
    print("Best hyperparameters:")
    print(grid_search.best_params_)

    return best_estimator, rmse, mape, r2


In [4]:
def evaluate_metrics(y_true, y_pred, label, target):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mean_target = np.mean(y_true)
    print(f"\n📊 {label} Set Performance for {target}:")
    print(f"Mean of y_{label.lower()}: {mean_target:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}")
    print(f"R²: {r2:.4f}")
    return rmse, mape, r2

In [5]:


def main():
    for target in ["AM Transparency", "PM Transparency"]:
        print(f"\n🚀 Evaluating model for: {target}")

        if target == "AM Transparency":
            X_train, X_dev, X_test, y_train, y_dev, y_test = prepare_am_transparency_data()
            model_filename = "../models/am_transparency_model.joblib"
        else:
            X_train, X_dev, X_test, y_train, y_dev, y_test = prepare_pm_transparency_data()
            model_filename = "../models/pm_transparency_model.joblib"

        best_model, dev_rmse, dev_mape, dev_r2 = evaluate_xgb(X_train, y_train, X_dev, y_dev)

        print("✅ Data Split Shapes:")
        print("  X_train:", X_train.shape)
        print("  X_dev:", X_dev.shape)
        print("  X_test:", X_test.shape)
        print("  y_train:", y_train.shape)
        print("  y_dev:", y_dev.shape)
        print("  y_test:", y_test.shape)

        y_train_pred = best_model.predict(X_train)
        y_dev_pred = best_model.predict(X_dev)
        y_test_pred = best_model.predict(X_test)

        evaluate_metrics(y_train, y_train_pred, "Train", target)
        evaluate_metrics(y_dev, y_dev_pred, "Dev", target)
        evaluate_metrics(y_test, y_test_pred, "Test", target)

        dump(best_model, model_filename)
        print(f"✅ Model saved as: {model_filename}")

if __name__ == "__main__":
    main()


🚀 Evaluating model for: AM Transparency
Evaluating XGBoost Regressor...
Fitting 5 folds for each of 18 candidates, totalling 90 fits

Top 10 Most Important Features:
                     Feature  Importance
3              num__Dec Rain    0.148040
17        cat__Season_Spring    0.125101
0       num__Spring Temp (F)    0.117636
5    num__Spring_Temp x Rain    0.117234
1          num__Max air temp    0.102456
2          num__Min air temp    0.094762
4           num__Calmar Rain    0.065631
8      num__Dec Rain (Lag 1)    0.051891
12     num__Dec Rain (Lag 3)    0.041917
6   num__Max Air Temp x Rain    0.039986
Grid searching is done!
Best score (neg MSE): -30.27994774756462
Best hyperparameters:
{'algo__learning_rate': 0.1, 'algo__max_depth': 4, 'algo__n_estimators': 1000, 'algo__subsample': 1.0}
✅ Data Split Shapes:
  X_train: (18632, 402)
  X_dev: (2329, 402)
  X_test: (2329, 402)
  y_train: (18632,)
  y_dev: (2329,)
  y_test: (2329,)

📊 Train Set Performance for AM Transparency:
Mea